In [1]:
from stereo3d import stereo_read_from_pickle as std_read
from parsivel import pars_read_from_pickle as pars_read
from pandas import DataFrame
from multifractal_analysis.spectral_analysis import get_beta_and_h
from multifractal_analysis.double_trace_moment import get_um_params_dtm
from multifractal_analysis.general import get_best_slice, fluctuations
from matplotlib import pyplot as plt
import numpy as np 
from pathlib import Path
pars_folder = Path("/home/debian-mmss/stage_project/data/saved_events/pars_events/")
stereo_folder = Path("/home/debian-mmss/stage_project/data/saved_events/stereo_events/")

Read all the saved events from parsivel and stereo 3d

In [2]:
# Read the main parsivel series
parsivel_events = [pars_read(file_path) for file_path in pars_folder.iterdir()]
stereo_events = [std_read(file_path) for file_path in stereo_folder.iterdir()]

Extract the rain rate from each series

In [13]:
data = [(stereo_event.rain_rate(30), pars_event.rain_rate)for stereo_event, pars_event in zip(stereo_events, parsivel_events)]
data = [(get_best_slice(str_rate, 2**6), get_best_slice(pars_rate, 2**6)) for str_rate, pars_rate in data]
data = [tup for tup in data if not (tup[0] is None or tup[1] is None)]
data = [(str_rate/np.mean(str_rate), pars_rate/np.mean(pars_rate)) for str_rate, pars_rate in data]
data = [(str_rate, pars_rate) for str_rate, pars_rate in data if not (any(np.isnan(pars_rate)) or any(np.isnan(str_rate)))]
len(data)

11

Spectral Analysis

In [14]:
results = []
problems = []
for str_rate, pars_rate in data:
    try:
        beta_str, h_str= get_beta_and_h(str_rate)
        beta_pars, h_pars = get_beta_and_h(pars_rate)
        alpha_str, c1_str = get_um_params_dtm(str_rate, 1.5)
        alpha_pars, c1_pars = get_um_params_dtm(pars_rate, 1.5)
        results.append((beta_str, h_str, alpha_str, c1_str, beta_pars, h_pars, alpha_pars, c1_pars))
    except:
        problems.append((str_rate, pars_rate))
        pass

results = DataFrame(data= results, columns=["beta_str", "h_str", "alpha_str", "c1_str", "beta_pars", "h_pars", "alpha_pars", "c1_pars"])
results

,beta_str,h_str,alpha_str,c1_str,beta_pars,h_pars,alpha_pars,c1_pars
0,2.270745,0.674426,1.899583,0.040591,2.034479,0.564051,1.868249,0.049239
1,2.179421,0.716786,1.747331,0.139929,2.345935,0.771815,1.359897,0.125558
2,2.395517,0.793114,1.979754,0.096103,2.840424,1.070690,1.979474,0.151675
3,2.210722,0.620826,2.028531,0.015295,2.307456,0.674547,1.988701,0.020910
4,2.057791,0.572835,2.028522,0.043458,1.981000,0.547189,2.024966,0.056144
5,2.597662,0.818081,2.019836,0.019103,2.496029,0.768171,2.058490,0.019705
6,2.288374,0.681089,1.928343,0.037934,2.148990,0.609382,1.955022,0.035497
7,2.732054,0.994073,2.027583,0.126686,1.995665,0.639572,1.987526,0.142423
8,1.487146,0.287090,1.965242,0.044105,1.719166,0.410779,1.997285,0.051250
9,2.285485,0.673886,1.949709,0.031753,2.111830,0.592932,1.936163,0.037938


In [15]:
for str_rate, pars_rate in problems:
    plt.plot(str_rate)
    plt.plot(pars_rate)
    break

Calculate the fluctuations for each field

In [16]:
fluc_data = [(fluctuations(str_rate), fluctuations(pars_rate))for str_rate, pars_rate in data]

Spectral Analysis

In [17]:
results = []
problems = []
for str_rate, pars_rate in fluc_data:
    try:
        beta_str, h_str= get_beta_and_h(str_rate)
        beta_pars, h_pars = get_beta_and_h(pars_rate)
        alpha_str, c1_str = get_um_params_dtm(str_rate, 1.5)
        alpha_pars, c1_pars = get_um_params_dtm(pars_rate, 1.5)
        results.append((beta_str, h_str, alpha_str, c1_str, beta_pars, h_pars, alpha_pars, c1_pars))
    except:
        problems.append((str_rate, pars_rate))
        pass

results = DataFrame(data= results, columns=["beta_str", "h_str", "alpha_str", "c1_str", "beta_pars", "h_pars", "alpha_pars", "c1_pars"])
results

,beta_str,h_str,alpha_str,c1_str,beta_pars,h_pars,alpha_pars,c1_pars
0,1.085690,0.147183,1.817173,0.111900,0.923685,0.098121,1.852356,0.144216
1,1.082830,0.235004,1.704449,0.216630,1.071187,0.194420,1.563789,0.187275
2,0.932233,0.089279,1.648136,0.140751,1.092140,0.237238,1.822701,0.204594
3,1.423105,0.284276,1.854122,0.076907,0.535775,-0.141094,1.778691,0.099046
4,1.138705,0.187305,1.810135,0.126839,0.793349,0.061233,1.936063,0.168661
5,0.860375,0.010361,1.904412,0.083174,1.001791,0.073030,1.863172,0.076022
6,1.556840,0.377415,1.795201,0.107057,0.486246,-0.186099,1.752140,0.077796
7,1.231973,0.252330,1.975936,0.137616,0.925511,0.141485,1.928421,0.183722
8,1.688068,0.469272,1.865861,0.131853,1.141347,0.189820,1.948117,0.121552
9,0.708136,-0.045462,1.830114,0.107224,1.018498,0.130841,1.821199,0.130206
